In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from insync.db import ListDB
from insync.list import CompletionCommand, ListItem, ListItemContext, ListItemContextType, ListRegistry


In [3]:
descs = ['eggs', 'milk', 'bread', 'butter']

reg = ListRegistry()

for desc in descs:
    i = ListItem(desc, context=ListItemContext('grocery', ListItemContextType.CHECKLIST))
    reg.add(i)

milk_item = list(reg.items)[1]
cmd = CompletionCommand(milk_item.uuid, True)

print("\nList:\n",reg, sep='')
print("Cmd before doing:\n\t", cmd)
reg.do(cmd)
print("Cmd after doing:\n\t", cmd)
print("\nList:\n",reg, sep='')

reg.undo()
print("\nList:\n",reg, sep='')


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery

Cmd before doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=None)
Cmd after doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=False)

List:
2024-03-01 eggs @^grocery
x 2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery



# Save and restore from the db

In [ ]:
DB_FILE = Path("scratch.db")
DB_FILE.unlink(missing_ok=True)
db = ListDB(DB_FILE)
db.ensure_tables_created()
db.patch(reg)

In [ ]:
reg2 = db.load()

reg.add(ListItem("cheese", context=ListItemContext('grocery', ListItemContextType.CHECKLIST)))
print("\nOriginal:\n",reg, sep='')
print("\nRestored:\n",reg2, sep='')

In [7]:
import dataclasses

@dataclasses.dataclass
class OrderableListItem:
    sort_key: str
    value: int

class OrderableList:
    """test example of a list that can be ordered
    goint to benchmark the how fast the key lenth grows with the number of
    insertions and reorders
    """
    def __init__(self):
        self.items: list[OrderableListItem] = []

    def _next_key(self) -> str:
        """return the next key in the sequence
        Keys be like a,b,c,...z,za,zb,zc,...zz,zza,zzb,zzc,...zzz,zzza,zzzb,...
        """
        if not self.items:
            return 'aa'

        last_key = max(self.items, key=lambda x: x.sort_key).sort_key
        last_2char = last_key[-2]
        if last_2char == 'zz':
            return last_key + 'aa'
        else:
            return last_key[:-1] + last_key[:-1] + chr(ord(last_2char) + 1)

    def append(self, v: int) -> None:
        """append an item to the end of the list
        """
        self.items.append(OrderableListItem(self._next_key(), v))

    def sorted_items(self) -> list[OrderableListItem]:
        return sorted(self.items, key=lambda x: x.sort_key)

    def sanity_check_insertion_order(self) -> None:
        """check that the items are in order after intial insertions"""
        from itertools import pairwise
        for a, b in pairwise(self.items):
            if a.value > b.value:
                raise ValueError(f"Items are out of order: {a} > {b}")


ol = OrderableList()
for j in range(1000):
    ol.append(j)

results = ol.sorted_items()

import pandas as pd

df = pd.DataFrame(results)
df['sort_key_len'] = df['sort_key'].apply(len)

df


,sort_key,value,sort_key_len
0,aa,0,2
1,aaaab,2,5
2,aaaab,3,5
3,aaaab,4,5
4,aaaab,5,5
...,...,...,...
995,aaaab,996,5
996,aaaab,997,5
997,aaaab,998,5
998,aaaab,999,5
